In [33]:
#!/bin/bash

# Variables for directory and filenames
WORKDIR=$PWD/NGS_Workflow
REF_GENOME_DIR="${WORKDIR}/Reference_Genome"
tracking_DIR="${WORKDIR}/Fastq_Data"
TRIMMED_DIR="${WORKDIR}/Trimmed_Data"
RESULTS_DIR="${WORKDIR}/Results"
QC_DIR="${RESULTS_DIR}/QC"
REF_GENOME="${REF_GENOME_DIR}/S288C_reference_sequence_R64-4-1_20230830.tracking"
REF_GENOME_ARCHIVE="S288C_reference_genome_Current_Release.tgz"
REF_GENOME_URL="http://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz"

tracking_URL="https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2527nnn/GSM2527967/GSM2527967_PMD4a.fpkm_tracking.gz"
tracking=GSM2527967_PMD4a.fpkm_tracking.gz 

In [34]:
# Creating directories
mkdir -p "${WORKDIR}" "${REF_GENOME_DIR}" "${tracking_DIR}" "${TRIMMED_DIR}" "${RESULTS_DIR}" "${QC_DIR}"
cd "${WORKDIR}"

In [35]:
# Step 1: Setup and Data Download
# Downloading the reference genome if it does not exist or is empty
if [ ! -s "${REF_GENOME_DIR}/${REF_GENOME_ARCHIVE}" ]; then
    wget -P "${REF_GENOME_DIR}" $REF_GENOME_URL
    tar -xzvf "${REF_GENOME_DIR}/${REF_GENOME_ARCHIVE}" -C "${REF_GENOME_DIR}"
else
    echo "Reference genome archive already exists and is not empty."
fi

#find "${REF_GENOME_DIR}" -name "*reference*.fsa.gz" -exec echo genome file: {} \;
find "${REF_GENOME_DIR}" -name "*reference*.fsa.gz" -exec gunzip -d {} > "${REF_GENOME}" \;

# Downloading FASTQ files if they do not exist or are empty
if [ ! -s "${tracking_DIR}/${tracking}" ]; then
    wget -P "${tracking_DIR}" "${tracking_URL}"
else
    echo "tracking file already exists and is not empty."
fi


--2024-01-07 21:20:23--  http://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
Resolving downloads.yeastgenome.org (downloads.yeastgenome.org)... 52.12.181.3, 54.148.130.184, 54.148.109.74, ...
Connecting to downloads.yeastgenome.org (downloads.yeastgenome.org)|52.12.181.3|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://sgd-archive.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz [following]
--2024-01-07 21:20:27--  http://sgd-archive.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
Resolving sgd-archive.yeastgenome.org (sgd-archive.yeastgenome.org)... 52.92.227.107, 52.218.179.51, 52.92.227.43, ...
Connecting to sgd-archive.yeastgenome.org (sgd-archive.yeastgenome.org)|52.92.227.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21142292 (20M) [application/gzip

In [10]:
# Step 2: Quality Control with FastQC
tracking "${tracking_DIR}"/*.tracking.gz -o "${QC_DIR}"

Skipping '/home/nabilah/Documents/teaching_linux_bash/examples/ngs_workflow/NGS_Workflow/NGS_Workflow/Fastq_Data/*.fasta.gz' which didn't exist, or couldn't be read


In [18]:
# Step 3: Read Trimming using Trimmomatic
echo trimmomatic PE -phred33 \
"${tracking_DIR}/${tracking}" \
"${TRIMMED_DIR}/trimmed.tracking.gz" "${TRIMMED_DIR}/unpaired.tracking.gz" \
LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

trimmomatic PE -phred33 / /trimmed.fasta.gz /unpaired.fasta.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36


In [21]:
# Step 4: Read Alignment
# bwa index "${REF_GENOME}"
# echo bwa mem "${REF_GENOME}" "${TRIMMED_DIR}/trimmed_R1.fastq.gz" "${TRIMMED_DIR}/trimmed_R2.fastq.gz" > "${RESULTS_DIR}/aligned_reads.sam"
# Illumina paired-end reads shorter than ~70bp:
# bwa aln ref.fa read1.fq > read1.sai; bwa aln ref.fa read2.fq > read2.sai
# bwa sampe ref.fa read1.sai read2.sai read1.fq read2.fq > aln-pe.sam
bwa aln "${REF_GENOME}" "${tracking_DIR}/${tracking}" > "${RESULTS_DIR}/${tracking%.tracking.gz}.sai"
bwa sampe "${REF_GENOME}" \
"${RESULTS_DIR}/${tracking%.tracking.gz}.sai" \
"${tracking_DIR}/${tracking}" > "${RESULTS_DIR}/aligned_reads1.sam"

exit 1

bash: /.sai: Permission denied


bash: /aligned_reads1.sam: Permission denied
exit
Restarting Bash


In [13]:
# Step 5: Post-Alignment Processing
samtools view -bS "${RESULTS_DIR}/aligned_reads.sam" | samtools sort -o "${RESULTS_DIR}/sorted_aligned_reads.bam"
picard MarkDuplicates I="${RESULTS_DIR}/sorted_aligned_reads.bam" O="${RESULTS_DIR}/marked_duplicates.bam" M="${RESULTS_DIR}/marked_dup_metrics.txt"

[E::hts_open_format] Failed to open file "/aligned_reads.sam" : No such file or directory
samtools view: failed to open "/aligned_reads.sam" for reading: No such file or directory
samtools sort: failed to read header from "-"
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [14]:
# Step 6: Variant Calling
freebayes -f "${REF_GENOME_DIR}/${REF_GENOME}" "${RESULTS_DIR}/marked_duplicates.bam" > "${RESULTS_DIR}/variants.vcf"

bash: /variants.vcf: Permission denied


: 1

In [18]:
# Step 7: Quality Control Analysis with MultiQC
multiqc "${QC_DIR}" -o "${QC_DIR}"


Usage: multiqc [OPTIONS] <analysis directory>

Error: Invalid value for '<analysis directory>': Path '' does not exist.

This is MultiQC v1.12

For more help, run 'multiqc --help' or visit http://multiqc.info



: 2

In [10]:
# Step 8: Generating a Basic Plot for Variant Statistics with R
echo "
# Load the VCF file
library(vcfR)
vcf <- read.vcfR('${RESULTS_DIR}/variants.vcf')

# Basic variant statistics
variant_summary <- summary(vcf)

# Plotting
pdf('${RESULTS_DIR}/variant_stats_plot.pdf')
plot(variant_summary)
dev.off()
" > "${RESULTS_DIR}/variant_stats.R"
Rscript "${RESULTS_DIR}/variant_stats.R"

exit 1

# End of Workflow
echo "Workflow completed."


bash: /variant_stats.R: Permission denied


Fatal error: cannot open file '/variant_stats.R': No such file or directory
exit
Restarting Bash
